In [147]:
# import packages
import pandas as pd
# import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import datetime
import re
from sklearn.externals import joblib 
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from scipy.interpolate import interpn
from collections import defaultdict

In [148]:
# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [149]:
sales_means =pd.read_csv('./data/sales_means.csv', sep='|').drop(columns=["predicted_promotion"])
scaler = joblib.load('scaler.pkl') 
df_test = pd.read_csv('data/orders0206_test.csv', sep='|', parse_dates=['time'])
df_test['date'] = [d.date() for d in df_test['time']]
df_train = pd.read_csv('data/orders0206_train.csv', sep='|', parse_dates=['time'])
df_train['date'] = [d.date() for d in df_train['time']]
bundles = pd.read_csv('./data/bundles.csv', sep=',')

In [150]:
sales_means.head()

,count,weekGroup,itemID,customerRating,manufacturer,brand,category1,category3,dateFirstSell,soldWeekly,...,avg_salesPrice_lag_5,brand_avg_avg_salesPrice_lag_5,brand_sum_count_lag_5,brand_avg_count_lag_5,category1_avg_avg_salesPrice_lag_5,category1_sum_count_lag_5,category1_avg_count_lag_5,category3_avg_avg_salesPrice_lag_5,category3_sum_count_lag_5,category3_avg_count_lag_5
0,0.000414,1,1,0.876,1,0,0,0,1,False,...,0.000296,0.022205,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0
1,0.064817,2,1,0.876,1,0,0,0,1,False,...,0.000296,0.022205,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0
2,0.007248,3,1,0.876,1,0,0,0,1,False,...,0.000296,0.022205,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0
3,0.000621,4,1,0.876,1,0,0,0,1,False,...,0.000296,0.022205,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0
4,0.000207,5,1,0.876,1,0,0,0,1,False,...,0.000296,0.022205,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0


In [151]:
sales_means = sales_means.join(bundles.set_index("itemID"), on="itemID", how="left")

In [152]:
sales_means.sample(frac=1).reset_index(drop=True)
X_train = sales_means.tail(len(sales_means)-2000)#[(sales_means['weekGroup']<=9)]# & (sales_means['weekGroup']==10)]
X_cv =  sales_means.head(2000)#[sales_means['weekGroup']==10]
X_test = sales_means[sales_means['weekGroup']==11]
Y_train = X_train['count']
Y_cv = X_cv['count']
Y_test = X_test['count']
del X_train['count']
del X_cv['count']
del X_test['count']
del X_train['weekGroup']
del X_cv['weekGroup']
del X_test['weekGroup']

In [153]:
del X_train['itemID']
del X_cv['itemID']

In [154]:
from xgboost import XGBRegressor
model = XGBRegressor(
    booster = 'gbtree', 
    colsample_bytree = 0.8, 
    gamma = 0, 
    learning_rate = 0.08, 
    max_depth= 3, 
    min_child_weight= 1, 
    n_estimators= 200, 
    objective= 'reg:logistic', 
    seed= 20, 
    silent= 1, 
    subsample= 0.8)
model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_cv, Y_cv)], 
    verbose=True, 
    early_stopping_rounds = 10)

[0]	validation_0-rmse:0.45206	validation_1-rmse:0.45748
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:0.41636	validation_1-rmse:0.42139
[2]	validation_0-rmse:0.38365	validation_1-rmse:0.38836
[3]	validation_0-rmse:0.35363	validation_1-rmse:0.35793
[4]	validation_0-rmse:0.32609	validation_1-rmse:0.33006
[5]	validation_0-rmse:0.30078	validation_1-rmse:0.30446
[6]	validation_0-rmse:0.27751	validation_1-rmse:0.28087
[7]	validation_0-rmse:0.25612	validation_1-rmse:0.25920
[8]	validation_0-rmse:0.23645	validation_1-rmse:0.23915
[9]	validation_0-rmse:0.21837	validation_1-rmse:0.22076
[10]	validation_0-rmse:0.20173	validation_1-rmse:0.20382
[11]	validation_0-rmse:0.18643	validation_1-rmse:0.18812
[12]	validation_0-rmse:0.17236	validation_1-rmse:0.17375
[13]	validation_0-rmse:0.15941	validation_1-rmse:0.16050
[14]	validation_0-rmse:0.14751	validation_1-rmse:0.14

[141]	validation_0-rmse:0.02795	validation_1-rmse:0.01036
[142]	validation_0-rmse:0.02794	validation_1-rmse:0.01036
[143]	validation_0-rmse:0.02791	validation_1-rmse:0.01037
[144]	validation_0-rmse:0.02790	validation_1-rmse:0.01036
[145]	validation_0-rmse:0.02789	validation_1-rmse:0.01036
[146]	validation_0-rmse:0.02786	validation_1-rmse:0.01035
[147]	validation_0-rmse:0.02785	validation_1-rmse:0.01035
[148]	validation_0-rmse:0.02782	validation_1-rmse:0.01035
[149]	validation_0-rmse:0.02781	validation_1-rmse:0.01035
[150]	validation_0-rmse:0.02781	validation_1-rmse:0.01035
[151]	validation_0-rmse:0.02779	validation_1-rmse:0.01034
[152]	validation_0-rmse:0.02778	validation_1-rmse:0.01034
[153]	validation_0-rmse:0.02777	validation_1-rmse:0.01034
[154]	validation_0-rmse:0.02776	validation_1-rmse:0.01035
[155]	validation_0-rmse:0.02774	validation_1-rmse:0.01034
[156]	validation_0-rmse:0.02773	validation_1-rmse:0.01034
[157]	validation_0-rmse:0.02772	validation_1-rmse:0.01034
[158]	validati

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.08, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=0, num_parallel_tree=1,
             objective='reg:logistic', random_state=20, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=20, silent=1, subsample=0.8,
             tree_method=None, validate_parameters=False, verbosity=None)

In [155]:
import plotly_express as px
columns = X_train.columns
feature_importances = pd.DataFrame({'columns': columns,'importance':model.feature_importances_})
feature_importances = feature_importances.sort_values(by='importance',ascending=False)
px.bar(feature_importances,x='columns',y='importance')

In [156]:
def evaluate_result(y: dict, y_pred: dict):
    monetary_value = 0
    y_pred = defaultdict(int, y_pred)  # return prediction of 0 for items without prediction
    
    for item in set(y_pred).difference(set(y)):
        y[item] = 0  # make sure that all items for which a demand has been predicted are contained in the actual demands
    
    for item, demand in y.items():
        predicted_demand = y_pred[item]
        price = product_prices[item]
        monetary_value += price * min(demand, predicted_demand)
        if predicted_demand > demand:
            monetary_value -= .6 * price * (predicted_demand - demand)
            
    return monetary_value

In [157]:
df_info = pd.read_csv('data/infos.csv', sep='|', index_col='itemID')
df_items = pd.read_csv('data/items.csv', sep='|', index_col='itemID')
df_items.head()

,brand,manufacturer,customerRating,category1,category2,category3,recommendedRetailPrice
itemID,,,,,,,
1,0,1,4.38,1,1,1,8.84
2,0,2,3.00,1,2,1,16.92
3,0,3,5.00,1,3,1,15.89
4,0,2,4.44,1,2,1,40.17
5,0,2,2.33,1,1,1,17.04


In [158]:
# actual demand
y = df_test.groupby(by='itemID')['order'].sum().to_dict()

# baseline 1 (average demand of previous 14 days)
y_baseline1 = df_train[df_train['time'] >= '2018-05-19'].groupby(by='itemID')['order'].sum().to_dict()

# baseline 2 (average demand of previous half year)
total_orders = df_train.groupby(by='itemID')['order'].sum().to_dict()
total_observed_days = (df_train['time'].dt.normalize().max() - df_train['time'].dt.normalize().min()).days
y_baseline2 = {item: orders / total_observed_days * 14 for item, orders in total_orders.items()}  # 14-day avg. demand

df_info = pd.read_csv('data/infos.csv', sep='|', index_col='itemID')
df_items = pd.read_csv('data/items.csv', sep='|', index_col='itemID')
product_prices = df_info['simulationPrice'].to_dict()

In [159]:
# apply to unseen data
y_xgboost_all = dict()
for prod in X_test.itemID.unique():
    train_predict = pd.DataFrame(scaler.inverse_transform(pd.DataFrame(model.predict(X_test[X_test.itemID == prod].drop(['itemID'], axis=1))))).rename(columns={0:'predicted_count'})
    # train_predict["actual_count"] = test[test.itemID == prod]["order"].sum()
    y_xgboost_all[prod] = int(train_predict["predicted_count"].sum().round())
#     if prod in df_train.itemID.unique():
#         y_xgboost_all[prod] = int(train_predict["predicted_count"].sum().round())
#     else:
#         y_xgboost_all[prod] = 0

In [160]:
# perfect result
print(f'Perfect Result: {evaluate_result(y, y):.2f}')

# baseline 1
print(f'Baseline 1: {evaluate_result(y, y_baseline1):.2f}')

# baseline 2
print(f'Baseline 2: {evaluate_result(y, y_baseline2):.2f}')

# random forest
print(f'XGBoost: {evaluate_result(y, y_xgboost_all):.2f}')
# 953796.09

Perfect Result: 7895975.87
Baseline 1: -3727365.60
Baseline 2: -1672504.21
XGBoost: 28855.78
